In [2]:
#!pip install "ibm-watsonx-ai" | tail -n 1
#!pip install -U "langchain>=0.3,<0.4" | tail -n 1
#!pip install -U "langchain_ibm>=0.3,<0.4" | tail -n 1
#!pip install python-dotenv

In [3]:
import os
from dotenv import load_dotenv
import getpass

# Load environment variables from .env file
load_dotenv()

# Retrieve credentials
project_id = os.getenv("PROJECT_ID")
credentials = {
    "url": os.getenv("URL"),
    "apikey": os.getenv("APIKEY")
}

In [4]:
from ibm_watsonx_ai.foundation_models.utils.enums import ModelTypes

print([model.name for model in ModelTypes])

['FLAN_T5_XXL', 'FLAN_UL2', 'MT0_XXL', 'GPT_NEOX', 'MPT_7B_INSTRUCT2', 'STARCODER', 'LLAMA_2_70B_CHAT', 'LLAMA_2_13B_CHAT', 'GRANITE_13B_INSTRUCT', 'GRANITE_13B_CHAT', 'FLAN_T5_XL', 'GRANITE_13B_CHAT_V2', 'GRANITE_13B_INSTRUCT_V2', 'ELYZA_JAPANESE_LLAMA_2_7B_INSTRUCT', 'MIXTRAL_8X7B_INSTRUCT_V01_Q', 'CODELLAMA_34B_INSTRUCT_HF', 'GRANITE_20B_MULTILINGUAL', 'MERLINITE_7B', 'GRANITE_20B_CODE_INSTRUCT', 'GRANITE_34B_CODE_INSTRUCT', 'GRANITE_3B_CODE_INSTRUCT', 'GRANITE_7B_LAB', 'GRANITE_8B_CODE_INSTRUCT', 'LLAMA_3_70B_INSTRUCT', 'LLAMA_3_8B_INSTRUCT', 'MIXTRAL_8X7B_INSTRUCT_V01']


In [25]:
#model_id_1 = ModelTypes.FLAN_UL2.value
#model_id_2 = ModelTypes.FLAN_T5_XXL.value


model_id_1 = "ibm/granite-13b-instruct-v2" 
model_id_2 = "ibm/granite-20b-code-instruct" 

In [26]:
print(model_id_1)
print(model_id_2)

ibm/granite-13b-instruct-v2
ibm/granite-20b-code-instruct


In [36]:
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from ibm_watsonx_ai.foundation_models.utils.enums import DecodingMethods

parameters = {
    GenParams.DECODING_METHOD: DecodingMethods.SAMPLE.value,
    GenParams.MAX_NEW_TOKENS: 200,
    GenParams.MIN_NEW_TOKENS: 1,
    GenParams.TEMPERATURE: 0.5,
    GenParams.TOP_K: 50,
    GenParams.TOP_P: 1
}

In [28]:
from langchain_ibm import WatsonxLLM

llm_1 = WatsonxLLM(
    model_id=model_id_1,
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id,
    params=parameters
    )
llm_2 = WatsonxLLM(
    model_id=model_id_2,
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id
    )

/Users/durgaprasad/Desktop/_WEEKEND/DAY4-AI/env3_11/lib/python3.11/site-packages/ibm_watsonx_ai/foundation_models/utils/utils.py:436: LifecycleWarning: Model 'ibm/granite-20b-code-instruct' is in deprecated state from 2025-04-16 until 2025-07-17. IDs of alternative models: None. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warn(model_state_warning, category=LifecycleWarning)


In [29]:
from langchain import PromptTemplate

prompt_1 = PromptTemplate(
    input_variables=["topic"], 
    template="Generate a random question about {topic}: Question: "
)
prompt_2 = PromptTemplate(
    input_variables=["question"],
    template="Answer the following question: {question}",
)

In [30]:
from langchain.chains import LLMChain

chain_1 = LLMChain(llm=llm_1, prompt=prompt_1, output_key='question')
chain_2 = LLMChain(llm=llm_2, prompt=prompt_2, output_key='answer')

In [31]:
from langchain.chains import SequentialChain

qa = SequentialChain(chains=[chain_1, chain_2], input_variables=["topic"], output_variables=['question', 'answer'], verbose=True)

In [35]:
qa.invoke({"topic": "Indian cricket world cup 2011"})



> Entering new SequentialChain chain...

> Finished chain.


{'topic': 'Indian cricket world cup 2011',
 'question': 'Who will be the opposition for India in their opening match of the 2011 Cricket World Cup?',
 'answer': '\n\nIndia will play against Pakistan in their opening match of the 201'}